# DMS-view

In [1]:
# note: using gcreplay analysis results, including mutation incidence data
df_path = "../../gcreplay/analysis/output/mutations.dms_with_gctree_incidence.csv"
# df_path = "../results/final_variant_scores/final_variant_scores.csv"

In [2]:
import pandas as pd

In [3]:
dms_df = pd.read_csv(df_path, index_col="mutation", dtype=dict(position_IMGT=pd.Int16Dtype()))
# remove linker sites
dms_df = dms_df[dms_df.chain != "link"]
# add indicator for wildtype data
dms_df["WT"] = dms_df.wildtype == dms_df.mutant
assert dms_df.position_IMGT.max() < 1000
dms_df["site"] = [f"{chain}-{str(pos).zfill(3)}" for chain, pos in zip(dms_df.chain, dms_df.position_IMGT)]
dms_df

,target,wildtype,position,position_IMGT,chain,mutant,codon,single_nt,bind,delta_bind,...,n_bc_expr,n_libs_expr,psr,delta_psr,n_bc_psr,n_libs_psr,WT,site,gctree_incidence,S5F_mutation_rate
mutation,,,,,,,,,,,,,,,,,,,,,
E1(H)A,CGG_naive,E,1,1,H,A,GAG,True,10.48829,0.03110,...,19.0,2.0,6.78608,-0.07452,17.0,2.0,False,H-001,3,0.000123
E1(H)C,CGG_naive,E,1,1,H,C,GAG,False,10.44477,-0.01243,...,20.0,2.0,7.44234,0.58174,17.0,2.0,False,H-001,0,0.000000
E1(H)D,CGG_naive,E,1,1,H,D,GAG,True,10.42771,-0.02948,...,16.0,2.0,6.95940,0.09880,13.0,2.0,False,H-001,13,0.000103
E1(H)E,CGG_naive,E,1,1,H,E,GAG,True,10.45719,0.00000,...,21170.0,2.0,6.86060,0.00000,18910.0,2.0,True,H-001,0,0.000000
E1(H)F,CGG_naive,E,1,1,H,F,GAG,False,10.51899,0.06180,...,24.0,2.0,6.91372,0.05313,21.0,2.0,False,H-001,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R124(L)S,CGG_naive,R,235,124,L,S,CGT,True,10.44756,-0.00963,...,39.0,2.0,6.93925,0.07866,33.0,2.0,False,L-124,0,0.000000
R124(L)T,CGG_naive,R,235,124,L,T,CGT,False,10.43023,-0.02696,...,36.0,2.0,6.88956,0.02896,34.0,2.0,False,L-124,0,0.000000
R124(L)V,CGG_naive,R,235,124,L,V,CGT,False,10.38231,-0.07488,...,27.0,2.0,6.78160,-0.07899,24.0,2.0,False,L-124,0,0.000000


In [4]:
site_df = (dms_df[['position', 'delta_bind', 'delta_expr', 'delta_psr', "gctree_incidence", "S5F_mutation_rate"]]
           .melt(id_vars=['position'], var_name='condition')
           # to avoid multiindex, apply functions to the 'series' "value" ([])
           # not a dataframe ([[]])
           .groupby(['position', 'condition'])['value']
           .agg(('mean', 'median', 'max', 'min'))
           .reset_index()
           .rename(columns={'position': 'site', 'mean': 'site_mean', 'median': 'site_median', 'max': 'site_max', 'min': 'site_min'})
)
site_df['condition'] = site_df['condition'].map({'delta_bind': 'binding',
                                                 'delta_expr': 'expression',
                                                 'delta_psr': 'polyspecificity',
                                                 'gctree_incidence': 'GC tree mutation incidence',
                                                 'S5F_mutation_rate': 'S5F mutation rate'})
site_df

,site,condition,site_mean,site_median,site_max,site_min
0,1,S5F mutation rate,0.000055,0.000000,0.000455,0.00000
1,1,binding,0.081294,0.039955,0.533650,-0.02948
2,1,expression,-0.029753,-0.026920,0.065510,-0.12703
3,1,polyspecificity,0.045481,0.023260,0.581740,-0.08146
4,1,GC tree mutation incidence,1.150000,0.000000,13.000000,0.00000
...,...,...,...,...,...,...
1095,235,S5F mutation rate,0.000000,0.000000,0.000000,0.00000
1096,235,binding,-0.002362,0.005305,0.047920,-0.07488
1097,235,expression,0.047940,0.052820,0.174100,-0.06369
1098,235,polyspecificity,0.008772,0.007980,0.243750,-0.14599


In [5]:
mut_df = (dms_df[['position', 'delta_bind', 'delta_expr', 'delta_psr', "gctree_incidence", 'S5F_mutation_rate', 'mutant']]
          .rename(columns={'position': 'site', 'mutant': 'mutation'}))
mut_df = mut_df.melt(id_vars=['site', 'mutation'],
                     var_name='condition',
                     value_name='mut_delta')
mut_df['condition'] = mut_df['condition'].map({'delta_bind': 'binding',
                                              'delta_expr': 'expression',
                                              'delta_psr': 'polyspecificity',
                                              'gctree_incidence': 'GC tree mutation incidence',
                                              'S5F_mutation_rate': 'S5F mutation rate'})
mut_df

,site,mutation,condition,mut_delta
0,1,A,binding,0.03110
1,1,C,binding,-0.01243
2,1,D,binding,-0.02948
3,1,E,binding,0.00000
4,1,F,binding,0.06180
...,...,...,...,...
21995,235,S,S5F mutation rate,0.00000
21996,235,T,S5F mutation rate,0.00000
21997,235,V,S5F mutation rate,0.00000
21998,235,W,S5F mutation rate,0.00000


In [6]:
meta_df = dms_df[['position', 'wildtype', 'chain', 'position_IMGT']].copy().drop_duplicates()
meta_df['label_site'] = meta_df[['chain', 'position_IMGT', 'wildtype']].apply(lambda x: f'{x[2]}{x[1]}({x[0]})', axis=1)
meta_df = meta_df[['position', 'chain', 'position_IMGT', 'wildtype', 'label_site']].rename(columns={'position': 'site', 'chain': 'protein_chain', 'position_IMGT': 'protein_site'})
meta_df

,site,protein_chain,protein_site,wildtype,label_site
mutation,,,,,
E1(H)A,1,H,1,E,E1(H)
V2(H)A,2,H,2,V,V2(H)
Q3(H)A,3,H,3,Q,Q3(H)
L4(H)A,4,H,4,L,L4(H)
Q5(H)A,5,H,5,Q,Q5(H)
...,...,...,...,...,...
L120(L)A,231,L,120,L,L120(L)
E121(L)A,232,L,121,E,E121(L)
I122(L)A,233,L,122,I,I122(L)


In [7]:
dmsview_df = pd.merge(site_df, mut_df, how='outer').merge(meta_df, how='outer')
dmsview_df

,site,condition,site_mean,site_median,site_max,site_min,mutation,mut_delta,protein_chain,protein_site,wildtype,label_site
0,1,S5F mutation rate,0.000055,0.0,0.000455,0.0,A,0.000123,H,1,E,E1(H)
1,1,S5F mutation rate,0.000055,0.0,0.000455,0.0,C,0.000000,H,1,E,E1(H)
2,1,S5F mutation rate,0.000055,0.0,0.000455,0.0,D,0.000103,H,1,E,E1(H)
3,1,S5F mutation rate,0.000055,0.0,0.000455,0.0,E,0.000000,H,1,E,E1(H)
4,1,S5F mutation rate,0.000055,0.0,0.000455,0.0,F,0.000000,H,1,E,E1(H)
...,...,...,...,...,...,...,...,...,...,...,...,...
21995,235,GC tree mutation incidence,0.000000,0.0,0.000000,0.0,S,0.000000,L,124,R,R124(L)
21996,235,GC tree mutation incidence,0.000000,0.0,0.000000,0.0,T,0.000000,L,124,R,R124(L)
21997,235,GC tree mutation incidence,0.000000,0.0,0.000000,0.0,V,0.000000,L,124,R,R124(L)
21998,235,GC tree mutation incidence,0.000000,0.0,0.000000,0.0,W,0.000000,L,124,R,R124(L)


In [8]:
dmsview_df.to_csv("dms-view.csv")